In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')/255.0
        self.test_x_f1 = np.load('../testImage_canny.npy')/255.0
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')/255.0
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')/255.0
        self.train_x_f3 = np.load('../trainImage_pyramid_G.npy')/255.0
        self.test_x_f3 = np.load('../testImage_pyramid_G.npy')/255.0

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, x_f3, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            print("CNN:x_f3",x_f3.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())
            
            #输入x_f3,卷积核为3*3 输出维度为32
            net1_4 = tf.layers.conv2d(inputs = x_f3,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_4'               # 命名用于获取变量
                                    )
            print("CNN:",net1_4.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.concat([net, net1_4], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')
        self.x_f3 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f3.shape[1], 
                                             self.train_x_f3.shape[2], self.train_x_f3.shape[3]],
                          name='x_f3')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_set_f3 = self.train_x_f3[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f3 = shuffled_set_f3[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.x_f3: batch_x_f3,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_f1: batch_x_f1_test,
                                                                    self.x_f2: batch_x_f2_test,
                                                                    self.x_f3: batch_x_f3_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.x_f3: batch_x_f3_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [3]:
dataset = '4_Flowers_1s'
epoch = 80
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 20.13, total 80 epochs ptime: 1677.57
#  [*] Training finished!
#  [*] Best Epoch:  2 , Accuracy:  0.8000000715255737
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-2
#  [*] Finished testing Best Epoch: 2 , accuracy:  0.7999999523162842 !

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [

Epoch: [ 3] [  10/  40] time: 51.7147, loss: 0.01553097
Epoch: [ 3] [  11/  40] time: 52.1718, loss: 0.00496036
Epoch: [ 3] [  12/  40] time: 52.6308, loss: 0.03305558
Epoch: [ 3] [  13/  40] time: 53.0864, loss: 0.01324985
Epoch: [ 3] [  14/  40] time: 53.5402, loss: 0.05532375
Epoch: [ 3] [  15/  40] time: 54.0008, loss: 0.01021299
Epoch: [ 3] [  16/  40] time: 54.4562, loss: 0.03982088
Epoch: [ 3] [  17/  40] time: 54.9171, loss: 0.00249038
Epoch: [ 3] [  18/  40] time: 55.3795, loss: 0.12590083
Epoch: [ 3] [  19/  40] time: 55.8341, loss: 0.02508674
Epoch: [ 3] [  20/  40] time: 56.2944, loss: 0.08228170
Epoch: [ 3] [  21/  40] time: 56.7697, loss: 0.06711110
Epoch: [ 3] [  22/  40] time: 57.2347, loss: 0.15668145
Epoch: [ 3] [  23/  40] time: 57.6827, loss: 0.05203860
Epoch: [ 3] [  24/  40] time: 58.1477, loss: 0.04196929
Epoch: [ 3] [  25/  40] time: 58.6010, loss: 0.04490113
Epoch: [ 3] [  26/  40] time: 59.0596, loss: 0.04363130
Epoch: [ 3] [  27/  40] time: 59.5218, loss: 0.0

Epoch: [ 6] [  32/  40] time: 124.7337, loss: 0.01847840
Epoch: [ 6] [  33/  40] time: 125.1942, loss: 0.02681600
Epoch: [ 6] [  34/  40] time: 125.6512, loss: 0.00825420
Epoch: [ 6] [  35/  40] time: 126.0863, loss: 0.01322879
Epoch: [ 6] [  36/  40] time: 126.5524, loss: 0.00642363
Epoch: [ 6] [  37/  40] time: 126.9884, loss: 0.01094356
Epoch: [ 6] [  38/  40] time: 127.4364, loss: 0.01140053
Epoch: [ 6] [  39/  40] time: 127.8936, loss: 0.01223017
[6/80] - ptime: 19.8161 loss: 0.01977581 acc: 0.22000 lr: 0.00100000
Epoch: [ 7] [   0/  40] time: 131.0273, loss: 0.00048803
Epoch: [ 7] [   1/  40] time: 131.4618, loss: 0.00034508
Epoch: [ 7] [   2/  40] time: 131.9261, loss: 0.00025039
Epoch: [ 7] [   3/  40] time: 132.3550, loss: 0.00048388
Epoch: [ 7] [   4/  40] time: 132.7849, loss: 0.00401493
Epoch: [ 7] [   5/  40] time: 133.2411, loss: 0.01372936
Epoch: [ 7] [   6/  40] time: 133.6938, loss: 0.00366623
Epoch: [ 7] [   7/  40] time: 134.1463, loss: 0.00305904
Epoch: [ 7] [   8/ 

Epoch: [10] [  11/  40] time: 198.0361, loss: 0.00304832
Epoch: [10] [  12/  40] time: 198.4906, loss: 0.00213001
Epoch: [10] [  13/  40] time: 198.9378, loss: 0.01308125
Epoch: [10] [  14/  40] time: 199.3997, loss: 0.00066128
Epoch: [10] [  15/  40] time: 199.8517, loss: 0.00931496
Epoch: [10] [  16/  40] time: 200.2977, loss: 0.00337142
Epoch: [10] [  17/  40] time: 200.7516, loss: 0.02574376
Epoch: [10] [  18/  40] time: 201.1994, loss: 0.00962032
Epoch: [10] [  19/  40] time: 201.6439, loss: 0.00671675
Epoch: [10] [  20/  40] time: 202.1055, loss: 0.00032256
Epoch: [10] [  21/  40] time: 202.5441, loss: 0.00007819
Epoch: [10] [  22/  40] time: 202.9898, loss: 0.00594761
Epoch: [10] [  23/  40] time: 203.4506, loss: 0.00073581
Epoch: [10] [  24/  40] time: 203.8826, loss: 0.00102231
Epoch: [10] [  25/  40] time: 204.3350, loss: 0.00560390
Epoch: [10] [  26/  40] time: 204.7843, loss: 0.00295673
Epoch: [10] [  27/  40] time: 205.2275, loss: 0.00092095
Epoch: [10] [  28/  40] time: 2

Epoch: [13] [  32/  40] time: 269.2797, loss: 0.00087940
Epoch: [13] [  33/  40] time: 269.7257, loss: 0.00249110
Epoch: [13] [  34/  40] time: 270.1869, loss: 0.00037189
Epoch: [13] [  35/  40] time: 270.6392, loss: 0.00017483
Epoch: [13] [  36/  40] time: 271.0874, loss: 0.00263151
Epoch: [13] [  37/  40] time: 271.5418, loss: 0.00312806
Epoch: [13] [  38/  40] time: 271.9871, loss: 0.00701457
Epoch: [13] [  39/  40] time: 272.4404, loss: 0.00542173
[13/80] - ptime: 19.8098 loss: 0.00442866 acc: 0.40000 lr: 0.00090000
Epoch: [14] [   0/  40] time: 275.9001, loss: 0.00008242
Epoch: [14] [   1/  40] time: 276.3443, loss: 0.00005812
Epoch: [14] [   2/  40] time: 276.7969, loss: 0.00089266
Epoch: [14] [   3/  40] time: 277.2465, loss: 0.00010779
Epoch: [14] [   4/  40] time: 277.6987, loss: 0.00065123
Epoch: [14] [   5/  40] time: 278.1491, loss: 0.00146313
Epoch: [14] [   6/  40] time: 278.5938, loss: 0.00010893
Epoch: [14] [   7/  40] time: 279.0396, loss: 0.00006196
Epoch: [14] [   8/

...

Epoch: [66] [   0/  40] time: 1361.6567, loss: 0.00000278
Epoch: [66] [   1/  40] time: 1362.1178, loss: 0.00000163
Epoch: [66] [   2/  40] time: 1362.5767, loss: 0.00000036
Epoch: [66] [   3/  40] time: 1363.0225, loss: 0.00000002
Epoch: [66] [   4/  40] time: 1363.4776, loss: 0.00002871
Epoch: [66] [   5/  40] time: 1363.9385, loss: 0.00000208
Epoch: [66] [   6/  40] time: 1364.4002, loss: 0.00000056
Epoch: [66] [   7/  40] time: 1364.8619, loss: 0.00000216
Epoch: [66] [   8/  40] time: 1365.3200, loss: 0.00000134
Epoch: [66] [   9/  40] time: 1365.7786, loss: 0.00000248
Epoch: [66] [  10/  40] time: 1366.2351, loss: 0.00000045
Epoch: [66] [  11/  40] time: 1366.6915, loss: 0.00000362
Epoch: [66] [  12/  40] time: 1367.1508, loss: 0.00000017
Epoch: [66] [  13/  40] time: 1367.6063, loss: 0.00002647
Epoch: [66] [  14/  40] time: 1368.0641, loss: 0.00000021
Epoch: [66] [  15/  40] time: 1368.5263, loss: 0.00000005
Epoch: [66] [  16/  40] time: 1368.9770, loss: 0.00000686
Epoch: [66] [ 

Epoch: [69] [  18/  40] time: 1433.3613, loss: 0.00000037
Epoch: [69] [  19/  40] time: 1433.8352, loss: 0.00000138
Epoch: [69] [  20/  40] time: 1434.2973, loss: 0.00003612
Epoch: [69] [  21/  40] time: 1434.7647, loss: 0.00000173
Epoch: [69] [  22/  40] time: 1435.2299, loss: 0.00000557
Epoch: [69] [  23/  40] time: 1435.6835, loss: 0.00001011
Epoch: [69] [  24/  40] time: 1436.1465, loss: 0.00000025
Epoch: [69] [  25/  40] time: 1436.6070, loss: 0.00000426
Epoch: [69] [  26/  40] time: 1437.0713, loss: 0.00000011
Epoch: [69] [  27/  40] time: 1437.5362, loss: 0.00000002
Epoch: [69] [  28/  40] time: 1437.9889, loss: 0.00000020
Epoch: [69] [  29/  40] time: 1438.4528, loss: 0.00001579
Epoch: [69] [  30/  40] time: 1438.9124, loss: 0.00000069
Epoch: [69] [  31/  40] time: 1439.3734, loss: 0.00001371
Epoch: [69] [  32/  40] time: 1439.8414, loss: 0.00000010
Epoch: [69] [  33/  40] time: 1440.2953, loss: 0.00000007
Epoch: [69] [  34/  40] time: 1440.7590, loss: 0.00000277
Epoch: [69] [ 

Epoch: [72] [  36/  40] time: 1505.9951, loss: 0.00000072
Epoch: [72] [  37/  40] time: 1506.4563, loss: 0.00000021
Epoch: [72] [  38/  40] time: 1506.9249, loss: 0.00000015
Epoch: [72] [  39/  40] time: 1507.3879, loss: 0.00000151
[72/80] - ptime: 20.2916 loss: 0.00000161 acc: 0.76000 lr: 0.00047830
Epoch: [73] [   0/  40] time: 1510.7336, loss: 0.00000287
Epoch: [73] [   1/  40] time: 1511.1909, loss: 0.00000003
Epoch: [73] [   2/  40] time: 1511.6565, loss: 0.00000012
Epoch: [73] [   3/  40] time: 1512.1194, loss: 0.00000996
Epoch: [73] [   4/  40] time: 1512.5740, loss: 0.00000001
Epoch: [73] [   5/  40] time: 1513.0463, loss: 0.00000045
Epoch: [73] [   6/  40] time: 1513.4972, loss: 0.00000011
Epoch: [73] [   7/  40] time: 1513.9630, loss: 0.00000011
Epoch: [73] [   8/  40] time: 1514.4258, loss: 0.00000070
Epoch: [73] [   9/  40] time: 1514.8810, loss: 0.00000958
Epoch: [73] [  10/  40] time: 1515.3372, loss: 0.00000685
Epoch: [73] [  11/  40] time: 1515.8092, loss: 0.00000126
Ep

Epoch: [76] [  13/  40] time: 1580.0800, loss: 0.00000044
Epoch: [76] [  14/  40] time: 1580.5322, loss: 0.00000401
Epoch: [76] [  15/  40] time: 1580.9940, loss: 0.00002262
Epoch: [76] [  16/  40] time: 1581.4558, loss: 0.00000785
Epoch: [76] [  17/  40] time: 1581.9091, loss: 0.00000019
Epoch: [76] [  18/  40] time: 1582.3685, loss: 0.00000081
Epoch: [76] [  19/  40] time: 1582.8340, loss: 0.00000006
Epoch: [76] [  20/  40] time: 1583.3009, loss: 0.00000013
Epoch: [76] [  21/  40] time: 1583.7632, loss: 0.00000038
Epoch: [76] [  22/  40] time: 1584.2246, loss: 0.00000012
Epoch: [76] [  23/  40] time: 1584.6886, loss: 0.00000478
Epoch: [76] [  24/  40] time: 1585.1584, loss: 0.00000045
Epoch: [76] [  25/  40] time: 1585.6126, loss: 0.00000019
Epoch: [76] [  26/  40] time: 1586.0709, loss: 0.00000089
Epoch: [76] [  27/  40] time: 1586.5351, loss: 0.00000006
Epoch: [76] [  28/  40] time: 1587.0048, loss: 0.00000020
Epoch: [76] [  29/  40] time: 1587.4762, loss: 0.00000077
Epoch: [76] [ 

Epoch: [79] [  31/  40] time: 1651.5826, loss: 0.00000014
Epoch: [79] [  32/  40] time: 1652.0426, loss: 0.00000022
Epoch: [79] [  33/  40] time: 1652.5070, loss: 0.00000295
Epoch: [79] [  34/  40] time: 1652.9661, loss: 0.00001453
Epoch: [79] [  35/  40] time: 1653.4537, loss: 0.00009426
Epoch: [79] [  36/  40] time: 1653.9160, loss: 0.00000002
Epoch: [79] [  37/  40] time: 1654.3848, loss: 0.00000870
Epoch: [79] [  38/  40] time: 1654.8459, loss: 0.00000144
Epoch: [79] [  39/  40] time: 1655.3042, loss: 0.00000084
[79/80] - ptime: 20.2702 loss: 0.00000436 acc: 0.74000 lr: 0.00047830
Epoch: [80] [   0/  40] time: 1658.5461, loss: 0.00000010
Epoch: [80] [   1/  40] time: 1659.0056, loss: 0.00000001
Epoch: [80] [   2/  40] time: 1659.4652, loss: 0.00000009
Epoch: [80] [   3/  40] time: 1659.9326, loss: 0.00000060
Epoch: [80] [   4/  40] time: 1660.3877, loss: 0.00000100
Epoch: [80] [   5/  40] time: 1660.8460, loss: 0.00002365
Epoch: [80] [   6/  40] time: 1661.3140, loss: 0.00000047
Ep

In [4]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放